In [2]:
import pybullet as p
import pybullet_data
import time
import numpy as np
import gymnasium as gym
from gymnasium import spaces

pybullet build time: Jan 29 2025 23:16:28


In [3]:
class MazeCarEnv(gym.Env):
    metadata = {'render_modes': ['human'], "render_fps": 30}

    def __init__(self, render_mode=None):
        super().__init__()

        # --- Define Action Space (Discrete Example) ---
        # 0: Forward, 1: Left, 2: Right
        self.action_space = spaces.Discrete(3)
        self.action_velocities = {
            0: (10.0, 10.0), # Forward
            1: (10.0, 10.0), # Left
            2: (10.0, 1.0)  # Right
        }

        # --- Define Observation Space ---
        # Example: [car_x, car_y, car_yaw, target_x, target_y]
        # Needs careful normalization/scaling! Box space is common.
        # Define reasonable low/high bounds based on your expected maze size
        low = np.array([-10, -10, -np.pi, -10, -10], dtype=np.float32)
        high = np.array([10, 10, np.pi, 10, 10], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

        # --- PyBullet Setup ---
        self.render_mode = render_mode
        self.client = p.connect(p.DIRECT if render_mode is None else p.GUI)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        p.setGravity(0, 0, -9.81)
        self.planeId = p.loadURDF("plane.urdf", physicsClientId=self.client)
        # Define goal areas (use class attributes)
        self.goal_area_1 = np.array([5, 5]) # Store only XY
        self.goal_area_2 = np.array([5, -5])
        self.goal_radius = 0.5
        self.target_goal_pos = None # Will be set in reset()
        self.correct_goal_index = -1 # 0 for goal 1, 1 for goal 2
        # Visualize goals (only works in GUI mode)
        if p.getConnectionInfo()['connectionMethod'] == p.GUI:
            goal_visual_shape_1 = p.createVisualShape(p.GEOM_SPHERE, radius=self.goal_radius, rgbaColor=[0, 1, 0, 0.5])
            goal_visual_shape_2 = p.createVisualShape(p.GEOM_SPHERE, radius=self.goal_radius, rgbaColor=[1, 0, 0, 0.5])
            p.createMultiBody(baseVisualShapeIndex=goal_visual_shape_1, basePosition=[5, 5, 0.1])
            p.createMultiBody(baseVisualShapeIndex=goal_visual_shape_2, basePosition=[5, -5, 0.1])

        # Load car (ensure path is correct)
        self.start_pos = [0,0,0.1]
        self.start_orn = p.getQuaternionFromEuler([0,0,0])
        self.carId = p.loadURDF("/urdf/simple_two_wheel_car.urdf", self.start_pos, self.start_orn, physicsClientId=self.client)
        # FIND YOUR JOINT INDICES HERE (as done in Phase 1)
        self.left_wheel_joint_index = 1 # Replace with actual index
        self.right_wheel_joint_index = 0 # Replace with actual index

        self.step_counter = 0
        self.max_steps_per_episode = 5000 # Limit episode length

    def _get_obs(self):
        pos, orn_quat = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        euler = p.getEulerFromQuaternion(orn_quat)
        yaw = euler[2]
        # Return observation matching the defined space
        return np.array([pos[0], pos[1], yaw, self.target_goal_pos[0], self.target_goal_pos[1]], dtype=np.float32)

    def _get_info(self):
        # Optional: provide extra info not used for learning
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        dist_goal1 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_1)
        dist_goal2 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_2)
        return {"distance_goal1": dist_goal1, "distance_goal2": dist_goal2, "target_goal_index": self.correct_goal_index}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.step_counter = 0

        # Reset car position/orientation
        p.resetBasePositionAndOrientation(self.carId, self.start_pos, self.start_orn, physicsClientId=self.client)
        p.resetBaseVelocity(self.carId, linearVelocity=[0,0,0], angularVelocity=[0,0,0], physicsClientId=self.client)
        # Reset wheel velocities (important!)
        self._set_wheel_velocities(0, 0) # Stop wheels

        # --- Randomly choose the CORRECT goal for this episode ---
        self.correct_goal_index = self.np_random.integers(0, 2) # 0 or 1
        if self.correct_goal_index == 0:
            self.target_goal_pos = self.goal_area_1
        else:
            self.target_goal_pos = self.goal_area_2

        observation = self._get_obs()
        info = self._get_info()

        # Optional: Add goal visualization if rendering
        if self.render_mode == "human":
             self._render_frame() # Might need specific visualization logic here

        return observation, info

    def _set_wheel_velocities(self, left_vel, right_vel):
         p.setJointMotorControl2(bodyUniqueId=self.carId,
                                jointIndex=self.left_wheel_joint_index,
                                controlMode=p.VELOCITY_CONTROL,
                                targetVelocity=left_vel,
                                physicsClientId=self.client)
         p.setJointMotorControl2(bodyUniqueId=self.carId,
                                jointIndex=self.right_wheel_joint_index,
                                controlMode=p.VELOCITY_CONTROL,
                                targetVelocity=right_vel,
                                physicsClientId=self.client)

    def step(self, action):
        # Apply action (set wheel velocities)
        left_vel, right_vel = self.action_velocities[action]
        self._set_wheel_velocities(left_vel, right_vel)

        # Step simulation
        if action in [1, 2]:  # Left or Right turn
            for _ in range(100):  # Adjust the range to fine-tune the turn angle
                p.stepSimulation(physicsClientId=self.client)

        # Step simulation for forward movement
        else:
            p.stepSimulation(physicsClientId=self.client)

        self.step_counter += 1

        # Get observation, calculate reward, check termination/truncation
        observation = self._get_obs()
        info = self._get_info()

        # Check goal conditions
        car_pos, _ = p.getBasePositionAndOrientation(self.carId, physicsClientId=self.client)
        in_goal_1 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_1) < self.goal_radius
        in_goal_2 = np.linalg.norm(np.array(car_pos[:2]) - self.goal_area_2) < self.goal_radius

        terminated = False
        reward = -0.1 # Small penalty per step

        if in_goal_1:
            if self.correct_goal_index == 0:
                reward = 100.0 # Reached correct goal 1
                terminated = True
            else:
                reward = -50.0 # Reached incorrect goal 1
                terminated = True # End episode even if wrong goal is reached
        elif in_goal_2:
            if self.correct_goal_index == 1:
                reward = 100.0 # Reached correct goal 2
                terminated = True
            else:
                reward = -50.0 # Reached incorrect goal 2
                terminated = True

        # Check truncation (max steps exceeded)
        truncated = self.step_counter >= self.max_steps_per_episode

        if self.render_mode == "human":
            self._render_frame()

        # terminated: Episode ended successfully (goal) or unsuccessfully (wrong goal, crash - add later)
        # truncated: Episode ended due to external limit (time limit)
        return observation, reward, terminated, truncated, info

    def render(self):
        # PyBullet handles rendering if connected with p.GUI
        # This method might be needed for specific Gym requirements or offscreen rendering
        if self.render_mode == "human":
             # Usually PyBullet's GUI handles this, but you might add overlays here
             pass
        # Implement other modes like 'rgb_array' if needed
        return None # Or an image array

    def _render_frame(self):
         # Could add debug text, lines etc. using p.addUserDebugText, p.addUserDebugLine
         pass

    def close(self):
        p.disconnect(physicsClientId=self.client)

In [6]:
from stable_baselines3 import PPO # Or A2C, DQN for discrete actions
from stable_baselines3.common.env_checker import check_env


# Instantiate and check the environment
env = MazeCarEnv(render_mode=None) # Enable rendering during training
# It will check your defined spaces, reset/step methods for compatibility
try:
    check_env(env)
    print("Environment check passed!")
except Exception as e:
    print(f"Environment check failed: {e}")
    env.close()
    exit()


# Define the model (PPO is a good starting point for Box observations)
model = PPO("MlpPolicy", 
            env, 
            verbose=1, 
            tensorboard_log="./ppo_mazecar_tensorboard/",
            batch_size=64,
            learning_rate=0.0003,
            n_steps=4096,
)

Environment check passed!
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/ferdinand/masterthesis/ve_pybullet/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [4]:
# Train the model
from stable_baselines3.common.callbacks import EvalCallback

from stable_baselines3.common.vec_env import DummyVecEnv

eval_env = DummyVecEnv([lambda: MazeCarEnv(render_mode="Human")])
eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=10000,
                             deterministic=True, render=False)
model.learn(total_timesteps=200000, callback=eval_callback)
model.save("ppo_mazecar_model")
env.close()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RPL-P)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu


NameError: name 'model' is not defined

In [7]:
# --- To evaluate ---
eval_env.close()
eval_env = MazeCarEnv(render_mode="human") # Render during evaluation
model = PPO.load("ppo_mazecar_model", env=eval_env)

obs, info = eval_env.reset()
for _ in range(10000): # Max steps for evaluation
    action, _states = model.predict(obs, deterministic=True)
     # Convert action to the correct format if necessary
    if isinstance(action, np.ndarray) and action.size == 1:  # Discrete action
        action = action.item()  # Convert to scalar (e.g., 2 instead of [2])
    
    obs, reward, terminated, truncated, info = eval_env.step(action)
    time.sleep(1./480.)
    if terminated or truncated:
        print(f"Evaluation episode finished. Reached target goal: {info.get('target_goal_index')}, Reward: {reward}")
        obs, info = eval_env.reset() # Reset for next evaluation episode

eval_env.close()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RPL-P)
GL_VERSION=4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3


In [ ]:
p.resetSimulation()
p.disconnect()

error: Not connected to physics server.

XIO:  fatal IO error 62 (Timer expired) on X server ":0"
      after 148211 requests (148211 known processed) with 0 events remaining.


: 